In [3]:
from reference_sentences import retrieve_reference_dicts
from arxiv_num_extraction import get_arxiv_numbers_from_pdf
from retrieval import crossencoder_response, bm25_search
from paragraph import get_paragraphs
import os

reference_dicts = retrieve_reference_dicts('new.pdf')
arxiv_nums = get_arxiv_numbers_from_pdf('new.pdf')
paragraphs = get_paragraphs('new.pdf')


In [5]:
print(len(reference_dicts))

32


In [2]:
print(reference_dicts[2])

{'11': ['We employ a residual connection  around each of the two sub-layers, followed by layer normalization .'], '1': ['We employ a residual connection  around each of the two sub-layers, followed by layer normalization .']}


In [3]:
def create_arxiv_num_sentence_dict(reference_dicts, arxiv_nums, page_num):
    reference_dict = reference_dicts[page_num]
    key_nums = list(reference_dict.keys())
    new_dict = {}
    unfound_nums = []
    for key_num in key_nums:
        int_key = int(key_num) - 1
        arxiv_num = arxiv_nums[int_key]
        if arxiv_num is None:
            unfound_nums.append(key_num)
        else:
            new_dict[arxiv_num] = reference_dict[key_num]
    return new_dict, unfound_nums
    

d = create_arxiv_num_sentence_dict(reference_dicts, arxiv_nums, 2)
print(d)

({'1607.06450': ['We employ a residual connection  around each of the two sub-layers, followed by layer normalization .']}, ['11'])


In [11]:
def find_similarity(arxiv_dict, arxiv_nums, type='bm25'):
    out = []
    for key in arxiv_dict.keys():
        article_path = os.path.join('pdf_save', key + '.pdf')
        corpus = get_paragraphs(article_path)
        if key == "9":
            print(corpus)
        for sentence in arxiv_dict[key]:
            if type == 'bm25':
                closest = bm25_search(corpus, sentence)
            else:
                closest = crossencoder_response(corpus, sentence)
            arxiv_num = arxiv_nums.index(key)
            out.append((arxiv_num + 1, sentence, closest))

    return out

find_similarity(d[0], arxiv_nums, type='crossencoder')

[(1,
  'We employ a residual connection  around each of the two sub-layers, followed by layer normalization .',
  'we have also experimented with convolutional neural networks. in our preliminary experiments, we observed that layer normalization offers a speedup over the baseline model without normalization, but batch normalization outperforms the other methods. with fully connected layers, all the hidden units in a layer tend to make similar contributions to the ﬁnal prediction and re-centering and re- scaling the summed inputs to a layer works well. however, the assumption of similar contributions is no longer true for convolutional neural networks. the large number of the hidden units whose receptive ﬁelds lie near the boundary of the image are rarely turned on and thus have very different statistics from the rest of the hidden units within the same layer. we think further research is needed to make layer normalization work well in convnets.')]

In [9]:
def get_references_from_page(reference_dicts, arxiv_nums, page_num=1, type='bm25'):
    arxiv_dict, unfound_nums = create_arxiv_num_sentence_dict(reference_dicts, arxiv_nums, page_num)
    out = find_similarity(arxiv_dict, arxiv_nums, type=type)
    return out, unfound_nums

out, unfound_nums = get_references_from_page(reference_dicts, arxiv_nums, 5, type='crossencoder')

In [10]:
out

[(8,
  'There are many choices of positional encodings, learned and fixed .',
  'next we introduce a fully convolutional architecture for se- quence to sequence modeling. instead of relying on rnns to compute intermediate encoder states zand decoder states hwe use convolutional neural networks (cnn). 3.1. position embeddings'),
 (8,
  'We also experimented with using learned positional embeddings  instead, and found that the two versions produced nearly identical results (see Table 3 row (E)).',
  'next we introduce a fully convolutional architecture for se- quence to sequence modeling. instead of relying on rnns to compute intermediate encoder states zand decoder states hwe use convolutional neural networks (cnn). 3.1. position embeddings')]

In [12]:
print(unfound_nums)

['12']
